PyTorch上でモデルを実行し、Core ML上でのモデルの出力が合っているかを検証するためのデータを出力する

In [1]:
!pip install dlshogi==0.1.1

     |████████████████████████████████| 3.9 MB 8.0 MB/s 
     |████████████████████████████████| 3.3 MB 7.8 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch

In [4]:
device = torch.device("cpu")

In [5]:
class SingleInputModel(torch.nn.Module):
  def __init__(self, model):
    super().__init__()
    self.model = model
  def forward(self, x):
    return self.model(x[:, 0:62, :, :], x[:, 62:119, :, :])

In [6]:
from dlshogi.data_loader import DataLoader

In [7]:
train_dataloader = DataLoader(DataLoader.load_files(["/content/drive/My Drive/ShogiAIBookData/dlshogi_with_gct-001.hcpe"]), 32, device, shuffle=False)

In [8]:
from dlshogi.common import *
from dlshogi.network.policy_value_network import policy_value_network
from dlshogi import serializers
from dlshogi import cppshogi

In [9]:
model = policy_value_network("resnet10_swish", add_sigmoid=True)
model.set_swish(False)#swishをx*sigmoid(x)で計算するモード
serializers.load_npz("/content/drive/My Drive/ShogiAIBook/model/model_resnet10_swish-072", model, False)
si_model = SingleInputModel(model)
si_model.eval()

SingleInputModel(
  (model): PolicyValueNetworkAddSigmoid(
    (l1_1_1): Conv2d(62, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l1_1_2): Conv2d(62, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (l1_2): Conv2d(57, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (l2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l3): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l4): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l5): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l6): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l7): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l8): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l9): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padd

In [11]:
xs = []
moves = []
results = []
n_batch = 32
for x1, x2, move, result, value in train_dataloader:
  # x1 = [32, 62, 9, 9], x2 = [32, 57, 9, 9]
  x = torch.cat([x1, x2], dim=1)
  # x = [32, 119, 9, 9]
  pred_move, pred_result = si_model(x)
  # pred_mode: [32, 2187], pred_result: [32, 1]
  xs.append(x.detach().numpy())
  moves.append(pred_move.detach().numpy())
  results.append(pred_result.detach().numpy())
  if len(xs) >= n_batch:
    break

In [12]:
with open("/content/drive/My Drive/ShogiAIBook/model/SampleIO.bin", "wb") as f:
  for arys in [xs, moves, results]:
    d = np.concatenate(arys, axis=0).tobytes()
    f.write(d)